# SchNet S2EF training example

The purpose of this notebook is to demonstrate some of the basics of the Open Catalyst Project's (OCP) codebase and data. In this example, we will train a schnet model for predicting the energy and forces of a given structure (S2EF task). First, ensure you have installed the OCP ocp repo and all the dependencies according to the [README](https://github.com/Open-Catalyst-Project/ocp/blob/master/README.md).

Disclaimer: This notebook is for tutorial purposes, it is unlikely it will be practical to train baseline models on our larger datasets using this format. As a next step, we recommend trying the command line examples. 

## Imports

In [ ]:
import sys
import torch
import os
sys.path.append(r"D:\Train_thử\fairchem-tio2-s2ef")

from ocpmodels.trainers import ForcesTrainer
from ocpmodels import models
from ocpmodels.common import logger
from ocpmodels.common.utils import setup_logging
setup_logging()

c:\Users\admin\Downloads\Train_thử\fairchem-tio2-s2ef\ocpmodels\models\scn\spherical_harmonics.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd = torch.load(os.path.

In [2]:
# a simple sanity check that a GPU is available
if torch.cuda.is_available():
    print("True")
else:
    print("False")

False


## The essential steps for training an OCP model

1) Download data

2) Preprocess data (if necessary)

3) Define or load a configuration (config), which includes the following
   
   - task
   - model
   - optimizer
   - dataset
   - trainer

4) Train

5) Depending on the model/task there might be intermediate relaxation step

6) Predict

## Dataset

This examples uses the LMDB generated from the following [tutorial](http://laikapack.cheme.cmu.edu/notebook/open-catalyst-project/mshuaibi/notebooks/projects/ocp/docs/source/tutorials/lmdb_dataset_creation.ipynb). Please run that notebook before moving on. Alternatively, if you have other LMDBs available you may specify that instead.

In [3]:
# set the path to your local lmdb directory
# train_src = "s2ef"

In [4]:
from ocpmodels.datasets import TrajectoryLmdbDataset
import numpy as np
import copy

train_src = "s2ef"

train_dataset = TrajectoryLmdbDataset({"src": train_src})

energies = []
for data in train_dataset:
  energies.append(data.y)

mean = np.mean(energies)
stdev = np.std(energies)

c:\Users\admin\anaconda3\envs\tio2-s2ef\lib\site-packages\IPython\core\interactiveshell.py:3550: UserWarning: TrajectoryLmdbDataset is deprecated and will be removed in the future.Please use 'LmdbDataset' instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Define config

For this example, we will explicitly define the config; however, a set of default config files exists in the config folder of this repository. Default config yaml files can easily be loaded with the `build_config` util (found in `ocp/ocpmodels/common/utils.py`). Loading a yaml config is preferrable when launching jobs from the command line. We have included our best models' config files [here](https://github.com/Open-Catalyst-Project/ocp/tree/master/configs/s2ef).

**Task** 

In [5]:
task = {
    'dataset': 'trajectory_lmdb', # dataset used for the S2EF task
    'description': 'Regressing to energies and forces for DFT trajectories from OCP',
    'type': 'regression',
    'metric': 'mae',
    'labels': ['potential energy'],
    'grad_input': 'atomic forces',
    'train_on_free_atoms': True,
    'eval_on_free_atoms': True
}

**Model** - SchNet for this example

In [6]:
model = {
    'name': 'gemnet_t',
    "num_spherical": 7,
    "num_radial": 128,
    "num_blocks": 3,
    "emb_size_atom": 512,
    "emb_size_edge": 512,
    "emb_size_trip": 64,
    "emb_size_rbf": 16,
    "emb_size_cbf": 16,
    "emb_size_bil_trip": 64,
    "num_before_skip": 1,
    "num_after_skip": 2,
    "num_concat": 1,
    "num_atom": 3,
    "cutoff": 6.0,
    "max_neighbors": 50,
    "rbf": {"name": "gaussian"},
    "envelope": {
      "name": "polynomial",
      "exponent": 5,
    },
    "cbf": {"name": "spherical_harmonics"},
    "extensive": True,
    "otf_graph": True,
    "output_init": "HeOrthogonal",
    "activation": "silu",
    "scale_file": "D:\\Train_thử\\fairchem-tio2-s2ef\\configs\\s2ef\\all\\gemnet\\scaling_factors\\gemnet-dT.json",
    "direct_forces": True,
}

**Optimizer**

In [7]:
optimizer = {
    'batch_size': 1,         # originally 32
    'eval_batch_size': 1,    # originally 32
    'num_workers': 0,
    'lr_initial': 5.e-4,
    'optimizer': 'AdamW',
    'optimizer_params': {"amsgrad": True},
    'scheduler': "ReduceLROnPlateau",
    'mode': "min",
    'factor': 0.8,
    'patience': 3,
    'max_epochs': 1,         # used for demonstration purposes
    'force_coefficient': 100,
    'ema_decay': 0.999,
    'clip_grad_norm': 10,
    'loss_energy': 'mae',
    'loss_force': 'l2mae',
}

**Dataset**

For simplicity, `train_src` is used for all the train/val/test sets. Feel free to update with the actual S2EF val and test sets, but it does require additional downloads and preprocessing. If you desire to normalize your targets, `normalize_labels` must be set to `True` and corresponding `mean` and `stds` need to be specified. These values have been precomputed for you and can be found in any of the [`base.yml`](https://github.com/Open-Catalyst-Project/ocp/blob/master/configs/s2ef/20M/base.yml#L5-L9) config files.

In [8]:
# Dataset
dataset = [
  {'src': train_src,
   'normalize_labels': True,
   "target_mean": mean,
   "target_std": stdev,
   "grad_target_mean": 0.0,
   "grad_target_std": stdev
   }, # train set 
  {'src': train_src},
]

# dataset = [
# {'src': train_src, 'normalize_labels': False}, # train set 
# {'src': train_src}, # val set (optional)
# {'src': train_src} # test set (optional - writes predictions to disk)
# ]

**Trainer**

Use the `ForcesTrainer` for the S2EF and IS2RS tasks, and the `EnergyTrainer` for the IS2RE task 

In [9]:
trainer = ForcesTrainer(
    task=task,
    model=copy.deepcopy(model), # copied for later use, not necessary in practice.
    dataset=dataset,
    optimizer=optimizer,
    identifier="S2EF-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    # is_vis=False,
    print_every=5,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=True, # use PyTorch Automatic Mixed Precision (faster training and less memory usage),
)

amp: true
cmd:
  checkpoint_dir: ./checkpoints\2026-01-25-10-39-44-S2EF-example
  commit: 0ac55b8
  identifier: S2EF-example
  logs_dir: ./logs\tensorboard\2026-01-25-10-39-44-S2EF-example
  print_every: 5
  results_dir: ./results\2026-01-25-10-39-44-S2EF-example
  seed: 0
  timestamp_id: 2026-01-25-10-39-44-S2EF-example
dataset:
  grad_target_mean: 0.0
  grad_target_std: !!python/object/apply:numpy._core.multiarray.scalar
  - &id001 !!python/object/apply:numpy.dtype
    args:
    - f8
    - false
    - true
    state: !!python/tuple
    - 3
    - <
    - null
    - null
    - null
    - -1
    - -1
    - 0
  - !!binary |
    H+KjJnIHbUA=
  normalize_labels: true
  src: s2ef
  target_mean: !!python/object/apply:numpy._core.multiarray.scalar
  - *id001
  - !!binary |
    XsP4k1ppdsA=
  target_std: !!python/object/apply:numpy._core.multiarray.scalar
  - *id001
  - !!binary |
    H+KjJnIHbUA=
gpus: 0
logger: tensorboard
model: gemnet_t
model_attributes:
  activation: silu
  cbf:
    name:

c:\Users\admin\Downloads\Train_thử\fairchem-tio2-s2ef\ocpmodels\trainers\base_trainer.py:154: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler() if amp else None
c:\Users\admin\anaconda3\envs\tio2-s2ef\lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


2026-01-25 10:40:36 (INFO): Loaded GemNetT with 31671825 parameters.


2026-01-25 10:40:36 (WARNING): Model gradient logging to tensorboard not yet supported.


## Check the model

In [10]:
print(trainer.model)

OCPDataParallel(
  (module): GemNetT(
    (radial_basis): RadialBasis(
      (envelope): PolynomialEnvelope()
      (rbf): GaussianSmearing()
    )
    (cbf_basis3): CircularBasisLayer(
      (radial_basis): RadialBasis(
        (envelope): PolynomialEnvelope()
        (rbf): GaussianSmearing()
      )
    )
    (mlp_rbf3): Dense(
      (linear): Linear(in_features=128, out_features=16, bias=False)
      (_activation): Identity()
    )
    (mlp_cbf3): EfficientInteractionDownProjection()
    (mlp_rbf_h): Dense(
      (linear): Linear(in_features=128, out_features=16, bias=False)
      (_activation): Identity()
    )
    (mlp_rbf_out): Dense(
      (linear): Linear(in_features=128, out_features=16, bias=False)
      (_activation): Identity()
    )
    (atom_emb): AtomEmbedding(
      (embeddings): Embedding(83, 512)
    )
    (edge_emb): EdgeEmbedding(
      (dense): Dense(
        (linear): Linear(in_features=1152, out_features=512, bias=False)
        (_activation): ScaledSiLU(
      

## Train

In [11]:
trainer.train()

c:\Users\admin\Downloads\Train_thử\fairchem-tio2-s2ef\ocpmodels\trainers\forces_trainer.py:330: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.scaler is not None):
c:\Users\admin\anaconda3\envs\tio2-s2ef\lib\site-packages\torch\amp\autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


2026-01-25 10:40:41 (INFO): forcesx_mae: 8.87e+02, forcesy_mae: 1.59e+03, forcesz_mae: 2.54e+03, forces_mae: 1.67e+03, forces_cos: -6.02e-02, forces_magnitude: 3.53e+03, energy_mae: 3.43e+05, energy_force_within_threshold: 0.00e+00, loss: 2.87e+03, lr: 5.00e-04, epoch: 2.00e-04, step: 5.00e+00


KeyboardInterrupt: 

### Load Checkpoint
Once training has completed a `Trainer` class, by default, is loaded with the best checkpoint as determined by training or validation (if available) metrics. To load a `Trainer` class directly with a pretrained model, specify the `checkpoint_path` as defined by your previously trained model (`checkpoint_dir`):

In [ ]:
checkpoint_path = os.path.join(trainer.config["cmd"]["checkpoint_dir"], "checkpoint.pt")
checkpoint_path

'./checkpoints/2021-09-04-08-51-28-SchNet-example/checkpoint.pt'

In [ ]:
model = {
    'name': 'schnet',
    'hidden_channels': 1024, # if training is too slow for example purposes reduce the number of hidden channels
    'num_filters': 256,
    'num_interactions': 3,
    'num_gaussians': 200,
    'cutoff': 6.0
}

pretrained_trainer = ForcesTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="SchNet-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    is_vis=False,
    print_every=10,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

pretrained_trainer.load_checkpoint(checkpoint_path=checkpoint_path)

amp: false
cmd:
  checkpoint_dir: ./checkpoints/2021-09-04-08-51-28-SchNet-example
  commit: 98a06d8
  identifier: SchNet-example
  logs_dir: ./logs/tensorboard/2021-09-04-08-51-28-SchNet-example
  print_every: 10
  results_dir: ./results/2021-09-04-08-51-28-SchNet-example
  seed: 0
  timestamp_id: 2021-09-04-08-51-28-SchNet-example
dataset:
  normalize_labels: false
  src: s2ef
gpus: 1
logger: tensorboard
model: schnet
model_attributes:
  cutoff: 6.0
  hidden_channels: 1024
  num_filters: 256
  num_gaussians: 200
  num_interactions: 3
optim:
  batch_size: 16
  eval_batch_size: 8
  factor: 0.8
  force_coefficient: 100
  lr_initial: 0.0001
  max_epochs: 1
  mode: min
  num_workers: 8
  patience: 3
  scheduler: ReduceLROnPlateau
slurm: {}
task:
  dataset: trajectory_lmdb
  description: Regressing to energies and forces for DFT trajectories from OCP
  eval_on_free_atoms: true
  grad_input: atomic forces
  labels:
  - potential energy
  metric: mae
  train_on_free_atoms: true
  type: regre

2021-09-04 08:51:51 (WARNING): Model gradient logging to tensorboard not yet supported.


2021-09-04 08:51:51 (INFO): Loading checkpoint from: ./checkpoints/2021-09-04-08-51-28-SchNet-example/checkpoint.pt


## Predict

If a test has been provided in your config, predictions are generated and written to disk automatically upon training completion. Otherwise, to make predictions on unseen data a `torch.utils.data` DataLoader object must be constructed. Here we reference our test set to make predictions on. Predictions are saved in `{results_file}.npz` in your `results_dir`.

In [ ]:
# make predictions on the existing test_loader
predictions = pretrained_trainer.predict(pretrained_trainer.test_loader, results_file="s2ef_results", disable_tqdm=False)

2021-09-04 08:51:51 (INFO): Predicting on test.


device 0: 100%|██████████| 79/79 [00:01<00:00, 44.68it/s]

2021-09-04 08:51:53 (INFO): Writing results to ./results/2021-09-04-08-51-28-SchNet-example/s2ef_s2ef_results.npz


In [ ]:
energies = predictions["energy"]
forces = predictions["forces"]